In [12]:
import json
import pandas as pd

In [25]:
r = pd.read_json('arxiv_data/merged_14-01')
type(r)

pandas.core.frame.DataFrame

In [26]:
df=pd.DataFrame(r)
df.head(1)

,identifier,datestamp,setSpec,id,created,updated,authors,title,categories,comments,journal-ref,doi,abstract
0,oai:arXiv.org:0710.2155,2014-01-03,physics,0710.2155,2007-10-10,2011-07-19,"{'keyname': ['Elsayed'], 'forenames': ['Tarek ...",Interpreting Recoil For Undergraduate Students,physics.ed-ph physics.class-ph,To Appear in The Physics Teacher Magazine,"The Physics Teacher 50, 209 (2012)",10.1119/1.3694068,"In this paper, I outline some problems in th..."


In [30]:
def data_for_classification():
    my_dict = {}
    my_dict["title"] = list(dict(r)["title"])
    my_dict["categories"] = list(dict(r)["categories"])
    my_dict["abstract"] = list(dict(r)["abstract"])
    
    return my_dict

three=data_for_classification()

In [33]:
#len(three["categories"])
#3042

In [36]:
#type(three)
#dict

In [41]:
tit_list=three['title']
cat_list=three['categories']
abs_list=three['abstract']


In [42]:
#DOING A PREPROCESSING
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [62]:
tokenizer_tit=Tokenizer(oov_token='<00V>')
tokenizer_tit.fit_on_texts(tit_list)
word_index_tit=tokenizer_tit.word_index

In [100]:
#len(word_index_tit)

5999

In [64]:
sequences_tit=tokenizer_tit.texts_to_sequences(tit_list)
padded_tit=pad_sequences(sequences_tit,padding='post')
print(padded_tit[0])

[2834 1970    7 1971 1972    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


In [65]:
print(padded_tit.shape)

(3042, 39)


In [68]:
#CATEGORY
tokenizer_cat=Tokenizer(oov_token='<00V>')
tokenizer_cat.fit_on_texts(cat_list)
word_index_cat=tokenizer_cat.word_index
#print(word_index_cat)

In [69]:
sequences_cat=tokenizer_cat.texts_to_sequences(cat_list)
padded_cat=pad_sequences(sequences_cat,padding='post')
print(padded_cat[0])

[ 6 71  2  6 64  2  0  0  0  0  0  0  0  0  0  0  0]


In [71]:
print(padded_cat.shape)

(3042, 17)


In [76]:
# TRAIN & TEST

training_tit_list=tit_list[:2040]
testing_tit_list=tit_list[2040:]
training_cat_list=cat_list[:2040]
testing_cat_list=cat_list[2040:]
#training_size=2040
#testing_size=1000

In [78]:
# WE SHOULD REWRITE THE CODE TO BE SURE THAT OUR TOKENIZER DON`T KNOW THE TEST DATA!!
len(training_tit_list)

2040

In [90]:
tokenizer_tr=Tokenizer(num_words=10000,oov_token='<00V>')
tokenizer_tr.fit_on_texts(training_tit_list)
word_index_tr=tokenizer_tr.word_index

In [95]:
sequences_tr=tokenizer_tr.texts_to_sequences(training_tit_list)
padded_tr=pad_sequences(sequences_tr,padding='post',maxlen=39)

In [98]:
sequences_te=tokenizer_tr.texts_to_sequences(testing_tit_list)
padded_te=pad_sequences(sequences_te,padding='post',maxlen=39)

In [111]:
#MODEL

import tensorflow as tf

model=tf.keras.Sequential([
    tf.keras.layers.Embedding(5999,10,input_length=39),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [112]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [114]:
num_epoches=30
history=model.fit(padded_tr,training_cat_list,epoches=num_epoches,validation_data=(padded_te,testing_cat_list),verbose=2)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'str'>"})